In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install catboost
!pip3 install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.3 MB/s eta 0:00:00


In [3]:
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 29.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [4]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.6 MB/s eta 0:00:00


In [1]:
import unicodedata
from pathlib import Path

import catboost as cb
from optuna.integration import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from category_encoders import CountEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import TargetEncoder
import os

In [2]:

exp_name = 'exp00013'

outputs_path = f'/content/drive/MyDrive/Colab Notebooks/signate2023/{exp_name}/'

submission_dir = outputs_path + 'submissions/'
submission_path = submission_dir + f'submission_{exp_name}.csv'

model_dir = outputs_path + f'signate-models/'



for dir in [model_dir]:
    os.makedirs(dir, exist_ok=True)

In [3]:
INPUT_DIR = Path("/content/drive/MyDrive/Colab Notebooks/signate2023/")

In [4]:
train_df = pd.read_csv(INPUT_DIR / "train.csv")
test_df = pd.read_csv(INPUT_DIR / "test.csv")
sub_df = pd.read_csv(INPUT_DIR / "submit_sample.csv", names=["id", "price"])

In [5]:
import unicodedata

# Define a function to normalize manufacturer names
def normalize_manufacturer_name(name):
    # Convert full-width characters to half-width
    name = unicodedata.normalize('NFKC', name)
    # Convert to lowercase
    name = name.lower()
    # Remove non-ASCII characters
    name = ''.join(char for char in name if ord(char) < 128)
    return name


class PreProcessTransformer(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # cylinderから数値を取り出す
        X["cylinders"] = X["cylinders"].astype(str).str.extract("(\d+)").astype("float32")

        # sizeの表記揺れを修正
        X["size"] = X["size"].str.replace("ー", "-").astype(str)
        X["size"] = X["size"].str.replace("−", "-").astype(str)

        # Apply the function to the 'manufacturer' column
        X['manufacturer'] = X['manufacturer'].apply(normalize_manufacturer_name)


        # Define a dictionary for manufacturer name mapping with special characters
        manufacturer_name_mapping = {
            'nisan': 'nissan',
            'toyot': 'toyota',
            'nisѕan': 'nissan',
            'subαru': 'subaru',
            'subru': 'subaru',
            'sαturn': 'saturn',
            'lexu':'lexus',
            'аcura': 'acura',
            'vоlkswagen': 'volkswagen',
            'vlkswagen': 'volkswagen',
            'lexuѕ': 'lexus',
            'ᴄhrysler': 'chrysler',
            'sturn': 'saturn'
        }

        # Replace the manufacturer names using the mapping
        X['manufacturer'] = X['manufacturer'].replace(manufacturer_name_mapping)


        # yearが3000年以降のものはおかしいので2000年に置換
        err_idx = X.query("year >= 3000").index
        X.loc[err_idx, "year"] = X.loc[err_idx, "year"].apply(lambda x: x - 1000)

        # 走行距離 / 製造年
        X["odometer_per_year"] = X["odometer"] / (2023 - X["year"])

        return X


class RankTransformer(TransformerMixin, BaseEstimator):
    """keyの中でvalueが何番目のものか"""

    def __init__(self, key: str, value: str):
        self.key = key
        self.value = value

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_new = X.copy()
        X_new[self.key] = X_new.groupby(self.key)[self.value].rank(method="dense")
        return X_new[self.get_feature_names_out()]

    def get_feature_names_out(self, input_features=None):
        return [self.key]


class OriginalTransformer(TransformerMixin, BaseEstimator):
    """数値特徴はそのまま、カテゴリ特徴はcategory型に変換"""

    def __init__(self, numeric_cols, categorical_cols):
        self.numeric_cols = numeric_cols
        self.categorical_cols = categorical_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_new = X.copy()

        # 数値変数
        X_new[self.numeric_cols] = X_new[self.numeric_cols].astype("float32")

        # カテゴリ変数
        #X_new[self.categorical_cols] = X_new[self.categorical_cols].astype("category")

        return X_new[self.get_feature_names_out()]

    def get_feature_names_out(self):
        return self.numeric_cols + self.categorical_cols


class CountTransformer(TransformerMixin, BaseEstimator):
    """CountEncoder"""

    def __init__(self):
        pass

    def fit(self, X, y=None):
        self.ce = CountEncoder(cols=X.columns.tolist(), handle_unknown=0)
        self.ce.fit(X)
        return self

    def transform(self, X):
        return self.ce.transform(X)

    def get_feature_names_out(self, input_features=None):
        return input_features


class AggTransformer(TransformerMixin, BaseEstimator):
    """集約特徴量"""

    def __init__(self, key, numeric_cols, agg_func: dict):
        self.key = key
        self.numeric_cols = numeric_cols
        self.agg_func = agg_func

    def fit(self, X, y=None):
        X = X.copy()
        X[self.key] = X[self.key].astype("category")
        self.agg_df = X.groupby(self.key)[self.numeric_cols].agg(self.agg_func)
        self.agg_df.columns = [f"{col}_{func}" for col, func in self.agg_df.columns.tolist()]

        return self

    def transform(self, X):
        X_new = pd.merge(X, self.agg_df, on=self.key, how="left")
        return X_new[self.get_feature_names_out()]

    def get_feature_names_out(self, input_features=None):
        return self.agg_df.columns.tolist()

In [7]:
train_df["price"] = np.log1p(train_df["price"])

In [8]:
train_df["price"]

0        10.225136
1         8.460623
2         9.299450
3         9.714383
4         8.548498
           ...    
27527    10.380125
27528     8.594339
27529    10.009108
27530     8.024535
27531     9.942804
Name: price, Length: 27532, dtype: float64

In [9]:
numeric_cols = ["year", "odometer", "odometer_per_year"]
categorical_cols = [
    'region',
    "cylinders",
    "manufacturer",
    "condition",
    "fuel",
    "title_status",
    "transmission",
    "drive",
    "size",
    "type",
    "paint_color",
    "state",
]

ct = ColumnTransformer(
    transformers=[
        (
            "ori",
            OriginalTransformer(numeric_cols, categorical_cols),
            categorical_cols + numeric_cols,
        ),
        (
            "tgt",
            TargetEncoder(target_type="continuous", random_state=88),
            [
                'region',
                "cylinders",
                "manufacturer",
                "condition",
                "fuel",
                "title_status",
                "transmission",
                "drive",
                "size",
                "type",
                "paint_color",
                "state",
            ],
        ),
        # *[
        #     (f"agg_{key}", AggTransformer(key, ["odometer"], {"mean"}), [key] + ["odometer"])
        #     for key in [
        #         "manufacturer",
        #     ]
        # ],
    ],
    verbose=True,
)
ct.set_output(transform="pandas")
pipe = Pipeline(
    steps=[
        ("preprocess", PreProcessTransformer()),
        ("ct", ct),
    ]
)
train_feat_df = pipe.fit_transform(train_df, train_df["price"])
test_feat_df = pipe.transform(test_df)


[ColumnTransformer] ........... (1 of 2) Processing ori, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing tgt, total=   0.1s


In [10]:
state_dic = {}

# データ数的に、重複は気にせず上書きしていっても大した時間にはならない
for i in train_feat_df.index:
    # ただし、欠損値で上書きされると困るので、そこだけケア
    if pd.isna(train_feat_df.at[i, 'ori__state'])==False:
        state_dic[train_feat_df.at[i, 'ori__region']] = train_feat_df.at[i, 'ori__state']

train_feat_df.loc[pd.isna(train_feat_df['ori__state']).index, 'ori__state'] = train_feat_df.loc[pd.isna(train_feat_df['ori__state']).index, 'ori__region'].map(state_dic)
test_feat_df.loc[pd.isna(test_feat_df['ori__state']).index, 'ori__state'] = test_feat_df.loc[pd.isna(test_feat_df['ori__state']).index, 'ori__region'].map(state_dic)

In [11]:
pd.set_option('display.max_columns', None)
train_feat_df.sample(5)

,ori__year,ori__odometer,ori__odometer_per_year,ori__region,ori__cylinders,ori__manufacturer,ori__condition,ori__fuel,ori__title_status,ori__transmission,ori__drive,ori__size,ori__type,ori__paint_color,ori__state,tgt__region,tgt__cylinders,tgt__manufacturer,tgt__condition,tgt__fuel,tgt__title_status,tgt__transmission,tgt__drive,tgt__size,tgt__type,tgt__paint_color,tgt__state
21482,2016.0,197242.0,28177.427734,syracuse,4.0,honda,excellent,gas,clean,automatic,fwd,compact,sedan,blue,ny,9.226957,9.027142,8.874482,9.342636,9.136663,9.194626,9.260645,8.956555,8.883911,8.972641,9.232511,9.326421
9039,2016.0,188545.0,26935.000000,denver,6.0,jeep,excellent,gas,clean,manual,4wd,mid-size,SUV,grey,co,9.137982,9.278189,9.333498,9.335798,9.133291,9.186800,9.065168,9.570226,9.163095,9.273770,9.163751,9.127444
24747,2011.0,43182.0,3598.500000,mcallen / edinburg,4.0,honda,like new,gas,clean,automatic,fwd,compact,sedan,black,tx,9.033587,9.018290,8.894480,9.417172,9.131509,9.185938,9.252561,8.949937,8.887744,8.969636,9.322976,9.216875
27514,2007.0,190877.0,11929.812500,dallas / fort worth,6.0,ram,excellent,gas,clean,automatic,4wd,full-size,pickup,silver,tx,9.008626,9.282309,9.869778,9.342636,9.136663,9.194626,9.260645,9.574360,9.325208,9.588571,9.048532,9.159868
17967,2012.0,139453.0,12677.545898,joplin,4.0,honda,fair,gas,rebuilt,automatic,fwd,mid-size,SUV,black,mo,8.833278,9.018286,8.887042,8.555744,9.130792,9.362057,9.253754,8.946212,9.152831,9.265543,9.326907,9.221510


In [12]:
from sklearn.preprocessing import LabelEncoder

# List of categorical columns to apply label encoding
categorical_cols = ['ori__region', 'ori__manufacturer', 'ori__condition', 'ori__fuel', 'ori__title_status',
                    'ori__transmission', 'ori__drive', 'ori__size', 'ori__type', 'ori__paint_color', 'ori__state']

# Create a copy of the dataframes
train_data_encoded = train_feat_df.copy()
test_data_encoded = test_feat_df.copy()

# Instantiate the LabelEncoder
le = LabelEncoder()

# Apply label encoding and missing value imputation
for col in categorical_cols:
    # Fill missing values with 'unknown'
    #train_data_encoded[col].fillna('unknown', inplace=True)
    #test_data_encoded[col].fillna('unknown', inplace=True)

    # Apply label encoding
    le.fit(pd.concat([train_data_encoded[col], test_data_encoded[col]], axis=0))
    train_data_encoded[col] = le.transform(train_data_encoded[col])
    test_data_encoded[col] = le.transform(test_data_encoded[col])

In [13]:
train_feat_df = train_data_encoded
test_feat_df = test_data_encoded

In [14]:
test_feat_df

,ori__year,ori__odometer,ori__odometer_per_year,ori__region,ori__cylinders,ori__manufacturer,ori__condition,ori__fuel,ori__title_status,ori__transmission,ori__drive,ori__size,ori__type,ori__paint_color,ori__state,tgt__region,tgt__cylinders,tgt__manufacturer,tgt__condition,tgt__fuel,tgt__title_status,tgt__transmission,tgt__drive,tgt__size,tgt__type,tgt__paint_color,tgt__state
0,2015.0,92553.0,11569.125000,357,4.0,7,0,2,0,0,1,1,0,8,5,9.163121,9.022139,9.328177,9.336846,9.133833,9.188075,9.254351,8.949574,9.318699,9.268543,9.075488,9.215693
1,2013.0,134385.0,13438.500000,271,4.0,29,3,2,5,0,1,2,9,0,37,9.087116,9.022139,8.971255,9.424397,9.133833,9.112537,9.254351,8.949574,9.157157,8.967961,9.329783,9.220844
2,2011.0,102489.0,8540.750000,2,4.0,37,2,2,0,0,1,1,9,0,35,9.141951,9.022139,8.813312,9.001304,9.133833,9.188075,9.254351,8.949574,9.318699,8.967961,9.329783,9.082106
3,2016.0,64310.0,9187.142578,74,6.0,19,0,0,0,0,0,2,0,8,5,9.128967,9.278021,9.333750,9.336846,9.893702,9.188075,9.254351,9.568117,9.157157,9.268543,9.075488,9.132006
4,1999.0,180839.0,7534.958496,134,8.0,14,0,2,4,0,0,2,0,9,27,9.145969,9.384104,8.884690,9.336846,9.133833,9.364663,9.254351,9.568117,9.157157,9.268543,9.038870,9.159737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,2016.0,90902.0,12986.000000,121,6.0,13,0,2,4,0,1,1,10,0,26,8.999279,9.278021,9.411440,9.336846,9.133833,9.364663,9.254351,8.949574,9.318699,9.623794,9.329783,9.336763
27533,2012.0,27234.0,2475.818115,174,4.0,29,0,2,4,0,1,2,9,10,33,9.321768,9.022139,8.971255,9.336846,9.133833,9.364663,9.254351,8.949574,9.157157,8.967961,9.454825,9.326568
27534,2002.0,99761.0,4750.523926,251,6.0,4,0,2,0,0,2,1,3,1,3,9.350021,9.278021,9.308718,9.336846,9.133833,9.188075,9.254351,9.383189,9.318699,9.076384,9.230831,9.191890
27535,2006.0,162279.0,9545.823242,60,6.0,4,0,2,0,0,0,2,9,9,5,9.233761,9.278021,9.308718,9.336846,9.133833,9.188075,9.254351,9.568117,9.157157,8.967961,9.038870,9.132006


In [15]:
def get_cv(df, n_fold=5):
    kf = KFold(n_splits=n_fold, shuffle=True, random_state=71)
    return list(kf.split(df))

# ref: https://www.guruguru.science/competitions/16/discussions/185c7dc6-5e3a-49c6-9c30-41bf007cc694/
def fit_lgbm(X, y, cv, categorical_cols: list = None, params: dict = None, verbose: int = 50):
    # パラメータがないときは、空の dict で置き換える
    if params is None:
        params = {}

    models = []
    n_records = len(X)
    oof_pred = np.zeros((n_records,), dtype=np.float32)

    for i, (idx_train, idx_valid) in enumerate(cv):
        x_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        x_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

        train_data = lgb.Dataset(data=x_train, label=y_train)
        test_data = lgb.Dataset(data=x_valid, label=y_valid)

        model = lgb.train(train_set=train_data,
            params=params,
            valid_sets=[train_data, test_data],
            categorical_feature=[],
            valid_names=['Train', 'Test'],
            early_stopping_rounds=5,
            verbose_eval=verbose,
        )

        pred_i = model.predict(x_valid)
        oof_pred[idx_valid] = pred_i
        models.append(model)

        # Convert predictions back to the original scale
        pred_i_original_scale = np.expm1(pred_i)
        oof_pred_original_scale = np.expm1(oof_pred[idx_valid])

        score = mean_absolute_percentage_error(np.expm1(y[idx_valid]), oof_pred_original_scale)
        print(f" - fold{i + 1} - {score:.4f}")

    score = mean_absolute_percentage_error(np.expm1(y), np.expm1(oof_pred))
    print("=" * 50)
    print(f"FINISHI: Whole Score: {score:.4f}")
    return score, oof_pred, models


def fit_cat(X, y, cv, categorical_cols: list = None, params: dict = None, verbose: int = 50):
    if params is None:
        params = {}

    models = []
    n_records = len(X)
    oof_pred = np.zeros((n_records,), dtype=np.float32)

    for i, (idx_train, idx_valid) in enumerate(cv):
        x_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        x_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

        model = cb.CatBoostRegressor(**params)
        model.fit(
            x_train,
            y_train,
            cat_features=categorical_cols,
            eval_set=[(x_valid, y_valid)],
            use_best_model=True,
            early_stopping_rounds=100,
            verbose=verbose,
        )
        pred_i = model.predict(x_valid)
        oof_pred[idx_valid] = pred_i
        models.append(model)
        score = mean_absolute_percentage_error(y_valid, pred_i)
        print(f" - fold{i + 1} - {score:.4f}")

    score = mean_absolute_percentage_error(y, oof_pred)

    print("=" * 50)
    print(f"FINISHI: Whole Score: {score:.4f}")
    return score, oof_pred, models


# optunaを使ったパラメータチューニング
# def tuning(train_feat_df, train_df, cv):
#     def objective(trial):
#         max_depth = trial.suggest_int("max_depth", 1, 10)
#         num_leaves = trial.suggest_int("num_leaves", 2, 2**max_depth)
#         colsample_bytree = trial.suggest_uniform("colsample_bytree", 0.1, 1.0)
#         subsample = trial.suggest_uniform("subsample", 0.1, 1.0)

#         params = {
#             "objective": "mape",
#             "n_estimators": 10000,
#             "learning_rate": 0.05,
#             "max_depth": max_depth,
#             "num_leaves": num_leaves,
#             "colsample_bytree": colsample_bytree,
#             "subsample": subsample,
#             "metric": "mape",
#             "importance_type": "gain",
#             "random_state": 88,
#         }

#         score, _, _ = fit_lgbm(
#             train_feat_df, train_df["price"], cv=cv, categorical_cols=[], params=params, verbose=-1
#         )
#         return score

#     study = optuna.create_study(direction="minimize")
#     study.optimize(objective, n_trials=100)
#     print("Number of finished trials:", len(study.trials))
#     print("Best trial:", study.best_trial.params)
#     return study.best_trial.params


def visualize_importance(models, feat_train_df):
    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df["feature_importance"] = model.feature_importances_
        _df["column"] = feat_train_df.columns
        _df["fold"] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], axis=0, ignore_index=True)

    order = (
        feature_importance_df.groupby("column")
        .sum()[["feature_importance"]]
        .sort_values("feature_importance", ascending=False)
        .index[:50]
    )

    fig, ax = plt.subplots(figsize=(12, max(6, len(order) * 0.25)))
    sns.boxenplot(
        data=feature_importance_df,
        x="feature_importance",
        y="column",
        order=order,
        ax=ax,
        palette="viridis",
        orient="h",
    )
    ax.tick_params(axis="x", rotation=90)
    ax.set_title("Importance")
    ax.grid()
    fig.tight_layout()
    return fig, ax


def visualize_oof_gt(oof, gt):
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(oof, gt, alpha=0.5)
    gt_max = gt.max()
    ax.plot(np.arange(0, gt_max), np.arange(0, gt_max), color="red", alpha=0.5, linestyle="--")
    ax.set_xlabel("Out Of Fold")
    ax.set_ylabel("Ground Truth")
    ax.grid()
    ax.legend()
    fig.tight_layout()

    fig, ax


def visualize_oof_pred(oof, pred):
    fig, ax = plt.subplots(figsize=(8, 6))

    bins = 100
    ax.hist(pred, bins=bins, density=True, alpha=0.5, label="Test")
    ax.hist(oof, bins=bins, density=True, alpha=0.5, label="OutOfFold")
    ax.grid()
    ax.legend()
    fig.tight_layout()

    fig, ax

In [17]:
train_df["price"]

0        10.225136
1         8.460623
2         9.299450
3         9.714383
4         8.548498
           ...    
27527    10.380125
27528     8.594339
27529    10.009108
27530     8.024535
27531     9.942804
Name: price, Length: 27532, dtype: float64

In [18]:
n_fold = 5
lgbm_params = {
    "objective": "rmse",  # または "regression", "regression_l1", "regression_l2" など
    "metric": "mape",  # または "l1", "l2", "rmse" など
    'deterministic':True,  # Ensure reproducibility
    'force_row_wise':True,  # Ensure reproducibility
    "random_state": 42,
    "verbosity": -1,
}

feat_cat_cols = train_feat_df.select_dtypes(include="category").columns.tolist()

cv = get_cv(train_feat_df, n_fold=5)

# training
score, oof, models = fit_lgbm(
    train_feat_df,
    y=train_df["price"],
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

[I 2023-08-01 14:54:59,228] A new study created in memory with name: no-name-49a8581d-2909-4104-8e1e-ac8e53793754

feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.

KeyboardInterrupt: ignored

In [ ]:
# inference
pred = np.array([model.predict(test_feat_df) for model in models])
pred = np.mean(pred, axis=0)
# Convert predictions back to the original scale
pred = np.expm1(pred)

# submission
sub_df["price"] = pred
sub_df.to_csv(f'{model_dir}/submission.csv', index=False, header=False)

In [ ]:
visualize_importance(models, train_feat_df)
visualize_oof_gt(oof, train_df['price'])
visualize_oof_pred(oof, pred)

AttributeError: ignored